In [4]:
import psycopg2

## Connect to the PostgreSQL database

In [50]:
def get_pg_connection(host, database, user, password):
    pg_conn = psycopg2.connect(host = host, database = database, user = user, password = password)
    return pg_conn
pg_conn = get_pg_connection('localhost', 'Capstone', 'postgres', 'Le@rn1ng')
pg_cur = pg_conn.cursor()

## Question 1
What are the 5 nearest ports to Singapore's JURONG ISLAND port? (country = 'SG',
port_name = 'JURONG ISLAND').Your answer should include the columns port_name and
distance_in_meters only.

In [42]:
sql_query = """
    CREATE TABLE nearest_ports_to_jurong_island AS
    SELECT "Main_port_name", "Wpi_country_code",
           ( 6371000 * ACOS(
        SIN(RADIANS(jurong_latitude_degrees) + (jurong_latitude_minutes / 60.0)) *
        SIN(RADIANS("Latitude_degrees") + ("Latitude_minutes" / 60.0)) +
        COS(RADIANS(jurong_latitude_degrees) + (jurong_latitude_minutes / 60.0)) *
        COS(RADIANS("Latitude_degrees") + ("Latitude_minutes" / 60.0)) *
        COS(RADIANS("Longitude_degrees") + ("Longitude_minutes" / 60.0) - RADIANS(jurong_longitude_degrees) - (jurong_longitude_minutes / 60.0)) 
        )) AS distance_in_meters
    FROM wpi_data, (SELECT "Latitude_degrees" AS jurong_latitude_degrees, "Latitude_minutes" AS jurong_latitude_minutes, "Longitude_degrees" AS jurong_longitude_degrees, "Longitude_minutes" AS jurong_longitude_minutes
                    FROM wpi_data
                    WHERE "Main_port_name" = 'JURONG ISLAND' AND "Wpi_country_code" = 'SG') AS jurong_island
    WHERE "Main_port_name" != 'JURONG ISLAND'
    ORDER BY distance_in_meters
    LIMIT 5
"""
pg_cur.execute(sql_query)

# Commit the changes to the database
pg_conn.commit()

## Question 2
Which country has the largest number of ports with a cargo_wharf? Your answer should
include the columns country and port_count only.

In [46]:
sql_query = """ 
    CREATE TABLE largest_num_of_ports AS
    SELECT "Wpi_country_code" AS country, COUNT(DISTINCT ("Main_port_name")) AS port_count 
    FROM wpi_data
    WHERE "Load_offload_wharves" = 'Y'
    GROUP BY country
    ORDER BY port_count DESC
    LIMIT 1
"""
pg_cur.execute(sql_query)

pg_conn.commit()

## Question 3
You receive a distress call from the middle of the North Atlantic Ocean. The person on
the line gave you a coordinates of lat: 32.610982, long: -38.706256 and asked for the nearest
port with provisions, water, fuel_oil and diesel. Your answer should include the columns
country, port_name, port_latitude and port_longitude only.

In [52]:
lat= 32.610982
long= -38.706256

sql_query = f""" 
    CREATE TABLE nearest_port_with_provvisions AS
    SELECT "Wpi_country_code" AS country,"Main_port_name" AS port_name, "Longitude_degrees" AS port_longitude, "Latitude_degrees" AS port_latitude 
    FROM wpi_data 
    WHERE "Supplies_provisions" = 'Y'AND "Supplies_water" = 'Y'AND "Supplies_fuel_oil" = 'Y'AND "Supplies_diesel_oil" = 'Y'
    ORDER BY (6371000 * ACOS(
        SIN(RADIANS({lat})) * SIN(RADIANS("Latitude_degrees")) +
        COS(RADIANS({lat})) * COS(RADIANS("Latitude_degrees")) *
        COS(RADIANS("Longitude_degrees") - RADIANS({long}))
    ))  
    LIMIT 1
"""
pg_cur.execute(sql_query)

pg_conn.commit()

## Close the cursor and connection

In [53]:
pg_cur.close()
pg_conn.close()